In [1]:
import math
import random
import string
import sys
import time
import numpy as np
import torch

import matplotlib.pyplot as plt
import src
from envs import build_env
#from src.evaluator import Evaluator
from src.slurm import init_signal_handler, init_distributed_mode
from t2.realtime_trainer import RealtimeTrainer
from src.utils import initialize_exp, str_diff
from t2.utils import get_parser, join_sai, to_sai_str

from t2.transformer import build_transformer
argv = [
        '--exp_name', 'first_train', 
        '--tasks', 'add_dataset', 
        '--n_enc_layers', '16', 
        '--n_heads', '4', 
        '--sinusoidal_embeddings', 'true', 
        '--num_workers', '4', 
        '--eval_onl', '0', 
        '--save_periodic', '0', 
        '--epoch_size', '10000', 
        '--batch_size', '32', 
        '--dropout', '0.1', 
        '--attention_dropout', '0.1', 
        '--emb_dim', '64', 
        '--bottleneck_dim', '64', 
        '--nn_output', '1', 
        '--input_seq_length', '47', 
        '--share_inout_emb', 'false'
]

parser = get_parser()

params = parser.parse_args(argv)
params.my_device='cuda'

print(params)

bs = params.batch_size
#dim = params.emb_dim

init_distributed_mode(params)
logger = initialize_exp(params)

# CPU / CUDA
if params.cpu:
    assert not params.multi_gpu
else:
    assert torch.cuda.is_available()
    
src.utils.CUDA = not params.cpu

env = build_env(params)

modules = build_transformer(env, params)
trainer = RealtimeTrainer(modules, env, params)


INFO - 01/08/23 22:44:11 - 0:00:00 - ============ Initialized logger ============
INFO - 01/08/23 22:44:11 - 0:00:00 - accumulate_gradients: 1
                                     amp: -1
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_early_stopping: True
                                     beam_eval: False
                                     beam_length_penalty: 1
                                     beam_size: 1
                                     bottleneck_dim: 64
                                     clip_grad_norm: 5
                                     command: python F:\Programs\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py '-f' 'C:\Users\Dmytro\AppData\Roaming\jupyter\runtime\kernel-657dd551-23fc-4593-8171-0748d559e9dc.json' --exp_id "dmytro_job"
                                     cpu: False
                                     debug: False
               

SAIEnvironment created
Namespace(dump_path='./dumped/', exp_name='first_train', save_periodic=0, exp_id='dmytro_job', fp16=False, amp=-1, emb_dim=64, bottleneck_dim=64, nn_output=1, input_seq_length=47, n_enc_layers=16, n_dec_layers=4, n_heads=4, dropout=0.1, attention_dropout=0.1, share_inout_emb=False, sinusoidal_embeddings=True, env_base_seed=0, max_len=512, batch_size=32, optimizer='adam,lr=0.0001', clip_grad_norm=5, epoch_size=10000, max_epoch=100000, stopping_criterion='', validation_metrics='', accumulate_gradients=1, num_workers=4, same_nb_ops_per_batch=False, reload_size=-1, env_name='sai', tasks='add_dataset', beam_eval=False, beam_size=1, beam_length_penalty=1, beam_early_stopping=True, reload_model='', reload_checkpoint='', eval_only=False, eval_verbose=0, eval_verbose_print=False, debug_slurm=False, debug=False, cpu=False, local_rank=-1, master_port=-1, my_device='cuda')
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank  

INFO - 01/08/23 22:44:11 - 0:00:00 - Found 370 parameters in model.
INFO - 01/08/23 22:44:12 - 0:00:01 - Optimizers: model


In [2]:
A = 'ABABAGALAMAGA'

print(trainer.single_act(A, A))

V93Z3AVZ<s>ZZZV


In [3]:
start_time = time.time()

losses = []
iterations_number = 1000

for it in range(iterations_number):
        for _ in range(params.batch_size):
            loss = trainer.learn(A, A, A, 1)
            
        print(it)

end_time = time.time()
elapsed_time = end_time - start_time
av_time = elapsed_time/iterations_number

print(f'Av time per iteration per trainer: {elapsed_time/iterations_number:.2f} seconds')


F:\workspace\ai\Transformers1\src\optim.py:72: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\python_arg_parser.cpp:1418.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=Z9V13FVZZZZZR, score=0.723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=VADZVAUZ<s>ZZZV, score=0.723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=ZZDZVAVZ<s>ZZZV, score=0.723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=V93Z2ZYZ<s>Z9RZ, score=0.723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=V93ZVZUZ<s>ZZZV, score=0.723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred

0


INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AA2ZVAUZZZAAA, score=0.7872340425531915
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=VZZZAAAZZZZZA, score=0.7872340425531915
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAABAAAZAZZAA, score=0.8723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AZAZAAAZZZAAA, score=0.851063829787234
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AZBK8AAZ<s>ZAZU, score=0.7872340425531915
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AZABAZY0<s>ZZZA, score=0.8297872340425532
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=ZZVKAAUZZZZZV, score=0.7446808510638298
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=VA2KAAARZZACA, score=0.8085106382978723
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=ZA</s>ZAFUZZZZZA, score=0.765957446808

1


INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAABAAAZAZAAA, score=0.8936170212765957
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAAKAAAAAZAAA, score=0.8723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAAKAAARAAARA, score=0.8723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAAKAAAAAZAAA, score=0.8723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAABAAAZAZZAA, score=0.8723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAABAAAAAAAAA, score=0.8936170212765957
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAAZAAAZAAZAA, score=0.851063829787234
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAAKAAAUAAAAA, score=0.8723404255319149
INFO - 01/08/23 22:44:14 - 0:00:03 - learning: src=ABABAGALAMAGA, pred=AAABAAAZAAZAA, score=0.8723404255319149
IN

...

INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - lea

123


INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:41 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:42 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:42 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:42 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:42 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:42 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:42 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:42 - 0:00:30 - learning: src=ABABAGALAMAGA, pred=ABABAGALAMAGA, score=1.0
INFO - 01/08/23 22:44:42 - 0:00:30 - lea

124


KeyboardInterrupt: 

In [ ]:
torch.cuda.get_device_name(next(trainers[0].modules['transformer'].te1.parameters()).device.index)

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

#rnds = [gen_rnd_chars(xy_data_size_const) for _ in range(100)]

print(trainers[0].act(A))
print(trainers[0].act(B))

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

losses = []
for it in range(2000):
    loss = trainer.learn(join_sai(A), join_sai(A))
    if loss[0]:
        l = loss[1].item()
        losses += [l]

plt.plot(losses)
plt.title('los s vs iteration')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()
